In [3]:
import os
import argparse
import time
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits import mplot3d
import torch
import torch.nn as nn        #for all Neural Network stuff
import torch.optim as optim  #for all optimizer stuff

In [4]:
#method argument not needed??
data_size = 2000 #number of points in linspace
batch_time = 20 #number of points in the interval after a random starting point
batch_size = 500 #number of intervals
niters = 2000 #number of iterations
test_freq = 1
viz = True
gpu = 1
adjoint = True

In [5]:
from torchdiffeq import odeint_adjoint as odeint

In [6]:
device = torch.device('cuda:' + str(gpu) if torch.cuda.is_available() else 'cpu')

In [7]:
#create linspace like usual
t = torch.linspace(0.0, 25.0, data_size)

sigma = 10.0
ro = 28.0
beta = 8.0/3.0

In [8]:
def random():
    rand = 0
    while abs(rand) < 1 or abs(rand) > 3:
        rand = np.random.normal(0, 1, 1)[0]
    return rand

In [9]:
def getSlopesTorch(x,function):
    dim =  x.shape[0]**3
    slopeList = torch.zeros((dim,3))
    i = 0
    for m in x:
        for l in x:
            for k in x:
                #print(slope([],[m,l,k]))
                slopeList[i] = function.forward([],torch.tensor([[m,l,k]]))
                i += 1
    return slopeList

In [10]:
#lambda is a nn.Module like Linear, Conv2D etc,,
class Lambda(nn.Module):
    def forward(self,t,u):
        x = u[0,0]
        y = u[0,1]
        z = u[0,2]

        DE = np.zeros_like(u[0])

        DE[0] = sigma*(y-x)
        DE[1] = x*(ro-z)-y
        DE[2] = x*y - beta*z

        return torch.tensor(DE)

In [11]:
def get_batch():
    s = torch.from_numpy(np.random.choice(np.arange(data_size-batch_time, dtype=np.int64), batch_size, replace=False))
#s is a torch tensor from a numpy array. the array is a random choice array, a rand number from 0 to ds-bt is chosen bs times.
#s has batch_size elements and each was equally likely to be chosen
#why ds-bt? why not just data_size as a rance since we are chosing from the t linspace with ds many points?
#it does ds-bt so that when it chooses the next bt number of points we will never fall out of the index range if
#    ds=1000 and bt=10 then then if i start at 990, I will end up with 990-1000 and not over.
    batch_y0 = true_y[s] #(M, D) <- idk what this means
#b_y0 is made of the randomly selected data points (dep vals) from true_y solution
    batch_t = t[:batch_time] #(T)
#b_t is the values rank 0 to b_time from your linspace t; the linspace values being looked at
    batch_y = torch.stack([true_y[s+i] for i in range(batch_time)],dim=0)  # (T, M, D)
    return batch_y0, batch_t, batch_y


In [12]:
def makedirs(dirname):
    if not os.path.exists(dirname):
        os.makedirs(dirname)

In [13]:
def drawDFT(ySpace, length, function):
    for l in ySpace:
        for k in ySpace:
            y = torch.tensor([[l,k]])
            t = torch.tensor([[]])
            #direction i and j + Magnitude of <i,j>
            i = function.forward(t,y)[:,0][0].numpy()
            j = function.forward(t,y)[:,1][0].numpy()
            magnitude = np.sqrt(i**2+j**2)
            
            #normalize and scale by h
            i = h*i/magnitude
            j = h*j/magnitude
            
            current_domain = np.linspace(l-(i/2), l+(i/2), 2) #the end points of the small intervals
            current_range = np.linspace(k-(j/2), k+(j/2), 2)

            plt.plot(current_domain, current_range, lw=0.2, color='b')
            plt.arrow(current_domain[0],current_range[0], i, j, color='b', lw=0.5, length_includes_head=True, head_length=0.1, head_width=0.1)

In [14]:
def visualize(y, color):
    
    #Draw True Y
    fig = plt.figure()
    ax = plt.axes(projection='3d')
    
    ax.plot3D(y.numpy()[:,0,0],y.numpy()[:,0,1],y.numpy()[:,0,2],color)
    plt.xlim(-30,30)
    plt.ylim(-30,30)
    ax.set_zlim(10,45)
    plt.savefig('png/{:03d}'.format(itr))
    plt.show()

In [15]:
def visualizeSl(Sl,color):
    
    slxT = torch.zeros(8000)
    slyT = torch.zeros(8000)
    slzT = torch.zeros(8000)
    
    for i in range(0,8000):
        slxT[i] = Sl[i,0]
        slyT[i] = Sl[i,1]
        slzT[i] = Sl[i,2]
    
    #Draw 
    fig = plt.figure()
    ax = plt.axes(projection='3d')

    ax.plot3D(slyT.numpy(),slxT.numpy(),slzT.numpy(),color)

    #drawDFNLorentz(Y.numpy(),h)
    plt.xlim(-2500,2500)
    plt.ylim(-2500,2500)
    ax.set_zlim(-2500,2500)
    plt.savefig('png/{:03d}'.format(itr))
    plt.show()

In [19]:
class ODEFunc(nn.Module): #create an ODE block as our neural network. usually multiple layers/modules
    
    def __init__(self):
        super(ODEFunc, self).__init__()
#self.net is the neural network layers one after the other L -> T -> L        
        self.net = nn.Sequential(    
            nn.Linear(3,50),
            nn.Tanh(),
            nn.Linear(50,100),
            nn.Tanh(),
            nn.Linear(100,50),
            nn.Tanh(),
            nn.Linear(50,3),
        )
        
        for m in self.net.modules():
            if isinstance(m,nn.Linear): #checks in the module m in self.net is an instance of the nn.Linear class
                nn.init.normal_(m.weight, mean=0, std=200.0) #for the Linear modules make all the weights randn with std 0.1
                nn.init.constant_(m.bias, val=0) #for the Linear modules make all the bias vector values 0
    
    def forward(self, t, y):
        return self.net(y) #essentially Lambda.forward(args) without the true_A >>>which is TBD<<<

In [20]:
class RunningAverageMeter(object): #Computes and stores the average and current value
    def __init__(self, momentum=0.99): #new object of this class will have mom of 0.99 and a reset val and avg
        self.momentum = momentum
        self.reset()
    def reset(self):                   #reset function called to set val and avg to None and 0 respectively
        self.val = None
        self.avg = 0
    def update(self, val):             #update is called externally to update the avg based on val
        if self.val is None:
            self.avg = val
        else:
            self.avg = (self.avg * self.momentum) + (val * (1 - self.momentum)) #reminds me of output error at L

In [ ]:
if __name__ == '__main__':
    
    #ii = 0
    X = torch.linspace(-45,45,45)
    
    func = ODEFunc()
    optimizer = optim.RMSprop(func.parameters())          #this is an optimizer designed to update parameters
    end = time.time()                                     #usually people use SGD but this is a mini-batch approach
                                                          #http://www.cs.toronto.edu/~tijmen/csc321/slides/lecture_slides_lec6.pdf
    #RunningAvgMeter Objects
    time_meter = RunningAverageMeter(0.97)
    loss_meter = RunningAverageMeter(0.97)
    
    #TRAINING
    for itr in range(1, niters+1):
        #initial conditions; requires 2D tensor
        true_y0 = torch.tensor([[random(),random(),random()]])
        
        with torch.no_grad(): #no_grad disables gradient finding when true_y is defined
            #print(torch.mm(true_y0**3,true_A))
            true_y = odeint(Lambda(),true_y0, t, method='adams') 
            #true_y is the solved ODE defined in Lambda
            #odeint only takes a function from nn.Module >> implications? see solver comparision Jupyter
            #it uses Lambda to provide the function to go solve the IVP given true_y0 with the ODE solver on t
            trueSl = getSlopesTorch(X.numpy(),Lambda())
        print("trueSl")
        
        optimizer.zero_grad() #set gradients to zero/ not computed
        
        #batch_y0, batch_t, batch_y = get_batch()
        #feeds odeint the ODEBlock, a set of y values based on random t values and the first ten values of t
        #pred_y = odeint(func, batch_y0, batch_t)
        #deviates from usual Loss of mean squared difference but sum in both cases is positive
        #print(pred_y.detach(),batch_y)
        #---------------------------------------
        predSl = getSlopesTorch(X.numpy(),func)
        print("predSl")
        
        #loss = torch.mean(torch.abs(pred_y-batch_y)**2) #ask Ivan about predY-bY, also this returns a number
        #--------------------------------------------------
        loss = torch.mean(torch.abs(predSl-trueSl)**2)
        print("loss")
        
        loss.backward() #backpropogation
        print("1")
        optimizer.step() #gradient descent and update parameters
        print("2")
        #Update time and loss meters
        time_meter.update(time.time()-end) #takes in time difference to establish velocity see RMS method details
        print("3")
        loss_meter.update(loss.item()) #the loss itself is a delta / change
        print("BPdone")
        with torch.no_grad():
            pred_y = odeint(func, true_y0, t)
            #loss = torch.mean(torch.abs(pred_y-true_y)**2)
            #-------------------------------------------------
            predSl = getSlopesTorch(X.numpy(),func)
            print("predSl2")
            
            loss = torch.mean(torch.abs(predSl-trueSl)**2)
            print("loss2")
            print('Iteration {:04d} | Total Loss {:.6f}'.format(itr, loss.item()))
        
            if itr % test_freq == 0:
                visualize(true_y,'b')
                visualize(pred_y,'m')
                visualizeSl(trueSl,'b')
                visualizeSl(predSl,'m')
                
        #torch.save(func.state_dict(), C:\Users\delis\Desktop\DEPython\modelP)
                
        end = time.time()

trueSl
predSl
loss
1
2
3
BPdone
